In [1]:
import sys
import pprint
import datetime
from tlslite.utils import keyfactory
import pymongo
import pprint
import math
import pandas as pd
import numpy as np
from plotly import tools
import plotly.graph_objs as go
import plotly.offline as py
import os

py.init_notebook_mode(connected=True)

In [3]:
client = pymongo.MongoClient('18.8.8.119')
cydb = client.cy_useract

issues = pd.DataFrame(list(cydb.useract_info.find()))
del issues['_id']
len(issues)
valid_data = issues.loc[issues['IMEI'].apply(lambda x:len(x.split(';')) == 2)]

272828

In [69]:
client = pymongo.MongoClient('18.8.8.209')
cydb = client.cy

topapps = pd.DataFrame(list(cydb.topapps.find(projection={'name': 1, 'package_name':1})))
del topapps['_id']

cy_packages = pd.DataFrame(list(cydb.cy_packages.find()))
del cy_packages['_id']

In [78]:
#获取包对应的应用名
total_apps = topapps.append(cy_packages)
total_apps_names = total_apps.set_index('package_name').to_dict()['name']

In [17]:
#获取3月，4月的数据
d318 = datetime.datetime(2018, 3, 18).timestamp()
d418 = datetime.datetime(2018, 4, 18).timestamp()
d518 = datetime.datetime(2018, 5, 18).timestamp()

df318 = valid_data[(valid_data['wallTime'] >= d318) & (valid_data['wallTime'] < d418)]
df418 = valid_data[(valid_data['wallTime'] >= d418) & (valid_data['wallTime'] < d518)]

In [239]:
imeis = len(df418['IMEI'].unique())

In [154]:
total_appclick418 = df418[df418.name == 'appclick']
total_appclick318 = df318[df318.name == 'appclick']

appclick418 = total_appclick418.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick418))
appclick318 = total_appclick318.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick318))

In [ ]:
top30_click = appclick418[-30:]
others = 1 - top30_click.sum()
top31_click = top30_click.append(pd.Series({'Others':others})).sort_values()

In [170]:
appclick_diff = appclick418.sub(appclick318, fill_value=0)[list(top30_click.index)]

In [211]:
package_name_diff = list(appclick_diff.index)
package_name_diff.reverse()

app_name_diff = []
for package in package_name_diff:
    if package in total_apps_names.keys():
        app_name_diff.append(total_apps_names[package])
    else:
        app_name_diff.append(package)

In [212]:
value_diff = list(appclick_diff.values)
value_diff.reverse()

In [225]:
packages_name = []
package_list = list(total_apps['package_name'])
for package in top31_click.index:
    if package in total_apps_names.keys():
        packages_name.append(total_apps_names[package])
    else:
        packages_name.append(package)

In [240]:
bar_data = {}
bar_data['y'] = packages_name
bar_data['x'] = top31_click.values
bar_data['text'] = top31_click.apply(lambda x:'{:.2f}%'.format(x*100)).values
table_data = [app_name_diff, ['{:.2f}%'.format(x*100) for x in value_diff]]

title = '点击次数排行榜<br>总点击数:{}  设备数:{}'.format(len(total_appclick418), imeis)
draw_bar_table(bar_data, table_data, height=1200, width=1300, title=title)

In [233]:
def draw_bar_table(bar_data, table_data, height, width, title):
    table_trace = go.Table(
        domain=dict(x=[0.45, 0.9],
                    y=[0, 0.8]),
        columnwidth = [52]*2,
        columnorder=[0, 1],
        header = dict(height = 25,
                    values = [['<b>名称</b>'],['<b>与上月同比</b>'],],
                    line = dict(color='#000000'),
                    align = ['left'] * 2,
                    font = dict(color=['#FFFFFF'] * 2, size=14),
                    fill = dict(color='#2d5b6c')),
        cells = dict(values = table_data,
                    line = dict(color='#000000'),
                    align = ['left'] * 2,
                    font = dict(color=['#FFFFFF'] * 2, size=12),
                    #format = [None]*4 + [',.2f'],
                    #prefix = [None] * 2 + ['$', u'\u20BF'],
                    #suffix=[None] * 4,
                    height = 25,
                    fill = dict(color=['#68a4b9', '#526372']))
    )

    trace = go.Bar(y=bar_data['y'], x=bar_data['x'], orientation = 'h',
                  text = bar_data['text'],textposition = 'outside',)

    layout = go.Layout(
        title=title,
        height=height,
        width=width,
        margin=dict(l=300,r=0),
        yaxis = dict(ticklen=2)
    )

    fig = go.Figure(data=[trace,table_trace],layout=layout)
    py.iplot(fig)


In [275]:
imeis418 = len(df418['IMEI'].unique())
click_by_phone418 = total_appclick418.groupby(['attrs_packagename']).size().sort_values().truediv(imeis418)
click_by_phone418 = click_by_phone418.apply(lambda x: round(x,1))

imeis318 = len(df318['IMEI'].unique())
click_by_phone318 = total_appclick318.groupby(['attrs_packagename']).size().sort_values().truediv(imeis318)
click_by_phone318 = click_by_phone318.apply(lambda x: round(x,1))

def get_bar_table_data(cur_month, last_month, rate=True, need_other=True):
    top30_click = cur_month[-30:]
    others = cur_month.sum() - top30_click.sum()
    
    if need_other:
        top31_click = top30_click.append(pd.Series({'Others':others})).sort_values()
    else:
        top31_click = top30_click
    appclick_diff = cur_month.sub(last_month, fill_value=0)[list(top30_click.index)]

    package_name_diff = list(appclick_diff.index)
    package_name_diff.reverse()

    app_name_diff = []
    for package in package_name_diff:
        if package in total_apps_names.keys():
            app_name_diff.append(total_apps_names[package])
        else:
            app_name_diff.append(package)
    
    value_diff = list(appclick_diff.values)
    value_diff.reverse()
    
    packages_name = []
    package_list = list(total_apps['package_name'])
    for package in top31_click.index:
        if package in total_apps_names.keys():
            packages_name.append(total_apps_names[package])
        else:
            packages_name.append(package)
    
    bar_data = {}
    bar_data['y'] = packages_name
    bar_data['x'] = top31_click.values
    if rate:
        bar_data['text'] = top31_click.apply(lambda x:'{:.2f}%'.format(x*100)).values
        table_data = [app_name_diff, ['{:.2f}%'.format(x*100) for x in value_diff]]
    else:
        bar_data['text'] = top31_click.apply(lambda x:round(x,1)).values
        table_data = [app_name_diff, [round(x,1) for x in value_diff]]
    
    
    return bar_data, table_data
    
bar_data, table_data = get_bar_table_data(click_by_phone418, click_by_phone318, False)

In [260]:
title='人均点击排名<br>总点击数:{}  设备数:{}'.format(len(total_appclick418), imeis418)

In [261]:
draw_bar_table(bar_data, table_data, height=1200, width=1300, title=title)

In [276]:
def install_rate(cur_month, last_month, label,title,height=1200,width=1200):
    data_cur = cur_month[cur_month.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    data_last = last_month[last_month.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    
    imeis_cur = len(cur_month['IMEI'].unique())
    imeis_last = len(last_month['IMEI'].unique())
    
    package_info_cur = data_cur.reset_index()
    total_cur = package_info_cur.groupby('attrs_packagename').size().sort_values().sum()
    package_rate_cur = package_info_cur.groupby('attrs_packagename').size().sort_values()/imeis_cur
    
    package_info_last = data_last.reset_index()
    total_last = package_info_last.groupby('attrs_packagename').size().sort_values().sum()
    package_rate_last = package_info_last.groupby('attrs_packagename').size().sort_values()/imeis_last
    
    bar_data, table_data = get_bar_table_data(package_rate_cur, package_rate_last, rate=True, need_other=False)
    
    title= title + '<br>总次数:{}  总设备数:{}'.format(str(int(total_cur)), imeis_cur)
 
    draw_bar_table(bar_data, table_data, height, width, title)

In [281]:
install_rate(df418, df318, 'appinstall', '安装应用覆盖率', height=1300,width=1300)

In [282]:
install_rate(df418, df318,'appremove', '卸载应用排行榜',height=1300,width=1300)

### Camera 应用 

In [284]:
chinese_name_photos = {'photo':'普通拍照', 'panorama':'全景', 'facebeauty':'美颜', 'hdr':'逆光', 'nightvideo':'夜景',
                'selfiepanorama':'全景自拍', 'polaroid':'心情照片','gifshot':'GIF动画','profession':'高级', 'arcfilter':'滤镜'}
chinese_name_videos = {'video':'普通摄像','slowmotion':'慢动作摄影', 'timelapse':'延时摄影'}

In [298]:
takephoto_data_cur = df418[df418['name'] == 'TakePhoto']
video_data_cur = df418[df418['name'] == 'Video']

total_camera_cur = len(takephoto_data_cur)
total_video_cur = len(video_data_cur)
total_used_cur = (len(takephoto_data_cur) + len(video_data_cur))

capturemode_cur = takephoto_data_cur.groupby(['attrs_Capturemode']).size().sort_values()

capture_dict_cur = {}
total_camera_dict_cur = {}
for p in chinese_name_photos:
    if p in capturemode_cur.index:
        capture_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
        total_camera_dict_cur[chinese_name_videos[p]] = capturemode_cur[p]
    else:
        total_camera_dict_cur[chinese_name_videos[p]] = 0
        capture_dict_cur[chinese_name_photos[p]] = 0

capture_chinese_data = pd.Series(capture_dict_cur).sort_values()

video_mode = video_data_cur.groupby(['attrs_Videomode']).size().sort_values()
video_dict_cur = {}
for p in chinese_name_videos:
    if p in video_mode.index:
        video_dict_cur[chinese_name_videos[p]] = video_mode[p]
        total_camera_dict_cur[chinese_name_videos[p]] = video_mode[p]
    else:
        capture_dict_cur[chinese_name_videos[p]] = 0
        total_camera_dict_cur[chinese_name_videos[p]] = 0

video_chinese_data = pd.Series(video_dict_cur).sort_values()

In [304]:
def get_camera_data(df):
    takephoto_data_cur = df[df['name'] == 'TakePhoto']
    video_data_cur = df[df['name'] == 'Video']

    total_camera_cur = len(takephoto_data_cur)
    total_video_cur = len(video_data_cur)
    total_used_cur = (len(takephoto_data_cur) + len(video_data_cur))

    capturemode_cur = takephoto_data_cur.groupby(['attrs_Capturemode']).size().sort_values()

    capture_dict_cur = {}
    total_camera_dict_cur = {}
    for p in chinese_name_photos:
        if p in capturemode_cur.index:
            capture_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
            total_camera_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
        else:
            total_camera_dict_cur[chinese_name_photos[p]] = 0
            capture_dict_cur[chinese_name_photos[p]] = 0

    capture_chinese_data = pd.Series(capture_dict_cur).sort_values()

    video_mode = video_data_cur.groupby(['attrs_Videomode']).size().sort_values()
    video_dict_cur = {}
    for p in chinese_name_videos:
        if p in video_mode.index:
            video_dict_cur[chinese_name_videos[p]] = video_mode[p]
            total_camera_dict_cur[chinese_name_videos[p]] = video_mode[p]
        else:
            capture_dict_cur[chinese_name_videos[p]] = 0
            total_camera_dict_cur[chinese_name_videos[p]] = 0

    video_chinese_data = pd.Series(video_dict_cur).sort_values()
    total_chinese_data = pd.Series(total_camera_dict_cur).sort_values()
    
    return [capture_chinese_data,video_chinese_data,total_chinese_data]

In [305]:
cur_camera = get_camera_data(df418)

In [309]:
last_camera = get_camera_data(df318)

In [312]:
last_camera[2].sum()

6034

In [55]:
trace = go.Bar(y=camera_data.index, x=camera_data.values, orientation = 'h',
                 text = camera_data.apply(lambda x:'{:.1f}%'.format(x*100/total_camera)).values,textposition = 'outside',)

layout = go.Layout(
    title='拍照使用排名<br>总使用次数:{}'.format(total_camera),
    height=400,
    width=1200,
    margin=dict(l=120),
    yaxis = dict(ticklen=10)
)

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

In [295]:
capture_dict_cur

{'GIF动画': 39,
 '全景': 154,
 '全景自拍': 104,
 '夜景': 75,
 '延时摄影': 23,
 '心情照片': 42,
 '慢动作摄影': 22,
 '普通拍照': 7913,
 '普通摄像': 478,
 '滤镜': 73,
 '美颜': 619,
 '逆光': 186,
 '高级': 0}

In [50]:

trace = go.Bar(y=video_chinese_data.index, x=video_chinese_data.values, orientation = 'h',
                  text = video_mode.apply(lambda x:'{:.1f}%'.format(x*100/total_video)).values,textposition = 'outside',)

layout = go.Layout(
    title='VideoMode排名<br>总使用次数:{}'.format(total_video),
    height=250,
    width=1200,
    margin=dict(l=120),
    yaxis = dict(ticklen=10)
)

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

In [51]:
camera_data = pd.Series(capture_dict).sort_values()

In [52]:
trace = go.Bar(y=camera_data.index, x=camera_data.values, orientation = 'h',
                 text = camera_data.apply(lambda x:'{:.1f}%'.format(x*100/total_used)).values,textposition = 'outside',)

layout = go.Layout(
    title='相机使用排名<br>总使用次数:{}'.format(total_used),
    height=400,
    width=1200,
    margin=dict(l=120),
    yaxis = dict(ticklen=10)
)

fig = go.Figure(data=[trace],layout=layout)
py.iplot(fig)

### 销量统计

In [ ]:
valid_data.groupby(['IMEI', 'wallTime']).size()

In [57]:
register = valid_data.groupby(['IMEI', 'registerTime']).size()

In [58]:
register_data = {}

In [59]:
for imei, group in register.reset_index().groupby(['IMEI']):
    register_date = datetime.datetime.fromtimestamp(group.iloc[-1]['registerTime'])
    register_data[imei] = register_date

In [60]:
psd = pd.DataFrame.from_dict(register_data, orient='index')

In [61]:
len(psd.index)

256

In [62]:
register_count = psd.groupby([0]).size()

In [63]:
cc = register_count.resample('D').sum().fillna(0)

In [64]:
paper_bgcolor='#242640'
plot_bgcolor='#242640'
paper_bgcolor='#FFFFFF'
plot_bgcolor='#FFFFFF'
text_color = '#000000'
x_color = '#000000'
y_color = '#000000'
title_color = '#000000'
legend_color = '#000000'

In [65]:
bandxaxis = go.XAxis(
        type='date',
        #range=[(created_by_day.index[start] - 1).strftime('%y-%m-%d'),
        #       (created_by_day.index[-1] + 1).strftime('%y-%m-%d'),],
        ticks="outside", 
        showticklabels=True,
        ticktext=list(cc.index.strftime('%m-%d')),
        tickvals=list(cc.index.strftime('%y-%m-%d')),
        tickfont=dict(size=11),
        #tickfont=dict(size=11,color='#B9B9C3'),
        ticklen=10,showgrid=True,
        #color='#FFFFFF'
    )

In [74]:
data=[go.Bar(
        x = cc.index,
        y = cc.values,
        text=cc.values, 
        textposition='outside',
        name = '每日提交数',)
        #marker=dict(color='#7D858C')), 
]


In [75]:
layout = go.Layout(
    title='销量趋势图<br>总数:' + str(imeis),
    titlefont=dict(color=title_color,size=25),
    height=1200,
    width=1600,
    bargap=0.1,
    xaxis = bandxaxis,
    yaxis = dict(ticklen=10),
    #bargroupgap=0.05,
    #paper_bgcolor=paper_bgcolor,
    #plot_bgcolor=plot_bgcolor,
    legend=dict(
        x=0.1,
        font=dict(size=16),
    )
)

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)